In [19]:
import pandas as pd

input_file = "US_Accidents_March23.csv"       # big raw file (same folder)
output_file = "US_Accidents_Cleaned.csv"      # cleaned file will be created here

chunksize = 100000
first = True

In [20]:
columns_to_drop = [
    "End_Time","End_Lat","End_Lng","Weather_Timestamp",
    "Amenity","Bump","Distance(mi)",
    "Precipitation(in)","Zipcode","Airport_Code"
]

In [21]:
important_cols = ["Severity", "City", "State", "Weather_Condition"]

In [22]:
numeric_cols = [
    "Severity",
    "Temperature(F)",
    "Humidity(%)",
    "Pressure(in)",
    "Visibility(mi)",
    "Wind_Speed(mph)"
]

In [23]:
def remove_outliers_iqr(df, cols):
    for col in cols:
        if col in df.columns:
            Q1 = df[col].quantile(0.25)
            Q3 = df[col].quantile(0.75)
            IQR = Q3 - Q1
            lower = Q1 - 1.5 * IQR
            upper = Q3 + 1.5 * IQR
            df = df[(df[col] >= lower) & (df[col] <= upper)]
    return df

In [24]:
print("Cleaning started... please wait.")

Cleaning started... please wait.


In [25]:
for chunk in pd.read_csv(input_file, chunksize=chunksize):

    # Convert Start_Time to datetime
    chunk["Start_Time"] = pd.to_datetime(chunk["Start_Time"], errors="coerce")

    # Feature Engineering
    chunk["Hour"] = chunk["Start_Time"].dt.hour
    chunk["Weekday"] = chunk["Start_Time"].dt.day_name()
    chunk["Month"] = chunk["Start_Time"].dt.month_name()

    # Drop original Start_Time
    chunk.drop(columns=["Start_Time"], inplace=True)

    # Drop unwanted columns
    chunk.drop(columns=columns_to_drop, inplace=True, errors="ignore")

    # Drop rows missing important values
    chunk.dropna(subset=important_cols, inplace=True)

    # Remove duplicates
    chunk.drop_duplicates(inplace=True)

    # Remove outliers
    chunk = remove_outliers_iqr(chunk, numeric_cols)

    # Save cleaned chunk
    mode = "w" if first else "a"
    chunk.to_csv(output_file, mode=mode, index=False, header=first)
    first = False

In [26]:
print("Cleaning completed!")
print("Cleaned file saved as:", output_file)

Cleaning completed!
Cleaned file saved as: US_Accidents_Cleaned.csv


In [27]:
df_clean = pd.read_csv(output_file, nrows=10)
df_clean.head()

,ID,Source,Severity,Start_Lat,Start_Lng,Description,Street,City,County,State,...,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight,Hour,Weekday,Month
0,A-3,Source2,2,39.063148,-84.032608,Accident on OH-32 State Route 32 Westbound at ...,State Route 32,Williamsburg,Clermont,OH,...,False,True,False,Night,Night,Day,Day,6,Monday,February
1,A-14,Source2,2,39.790760,-84.241547,Accident on Salem Ave at Hillcrest Ave / Kensi...,Salem Ave,Dayton,Montgomery,OH,...,False,True,False,Day,Day,Day,Day,8,Monday,February
2,A-18,Source2,2,39.752174,-84.239952,Accident on Delphos Ave at Brooklyn Ave. Expec...,Delphos Ave,Dayton,Montgomery,OH,...,False,False,False,Day,Day,Day,Day,9,Monday,February
3,A-20,Source2,2,39.790703,-84.244461,Accident on Hillcrest Ave at Piccadilly Ave. E...,W Hillcrest Ave,Dayton,Montgomery,OH,...,False,False,False,Day,Day,Day,Day,9,Monday,February
4,A-22,Source2,2,39.773346,-84.224686,Accident on Princeton Dr at Catalpa Dr. Expect...,Princeton Dr,Dayton,Montgomery,OH,...,False,False,False,Day,Day,Day,Day,10,Monday,February
